In [1]:
from sagemaker import get_execution_role
role = get_execution_role()

In [2]:
pip install awswrangler

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pygeohash

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import sagemaker
import boto3
import awswrangler as wr

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import pygeohash as pgh

In [5]:
import os
import boto3

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

import sagemaker
from sagemaker import get_execution_role
import sagemaker_pyspark

role = get_execution_role()

# Configure Spark to use the SageMaker Spark dependency jars
jars = sagemaker_pyspark.classpath_jars()

classpath = ":".join(sagemaker_pyspark.classpath_jars())

# See the SageMaker Spark Github to learn how to connect to EMR from a notebook instance
spark = (
    SparkSession.builder.config("spark.driver.extraClassPath", classpath)\
#    .config('spark.driver.memory','32G')\
    .master("local[*]")\
    .getOrCreate()
)

spark

In [6]:
# https://sagemaker-examples.readthedocs.io/en/latest/sagemaker-spark/pyspark_mnist/pyspark_mnist_kmeans.html#Loading-the-Data
import boto3

cn_regions = ["cn-north-1", "cn-northwest-1"]
region = boto3.Session().region_name
endpoint_domain = "com.cn" if region in cn_regions else "com"
spark._jsc.hadoopConfiguration().set(
    "fs.s3a.endpoint", "s3.{}.amazonaws.{}".format(region, endpoint_domain)
)

In [7]:
from pyspark.sql.functions import *

In [8]:
center_name = 'WASHINGTON' #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< CHOOSE!!!
hour_diff = 5 #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< CHOOSE!!!

time_interval = 6
time_lag = "INTERVAL " + str(hour_diff) + " HOURS"
time_lag

'INTERVAL 5 HOURS'

In [9]:
%%time
latlon = spark.read.format('csv')\
        .options(header= 'true', inferSchema= 'true')\
        .load("s3a://partly-cloudy-common-area/proof_of_concept/latlonRAP.csv")

latlon.count()

CPU times: user 2.1 ms, sys: 1.11 ms, total: 3.21 ms
Wall time: 5.72 s


67725

In [10]:
latlon.show(5)

+---+---+------------------+-------------------+
| Nx| Ny|               Lat|                Lon|
+---+---+------------------+-------------------+
|  1|  1|16.280999999999988|-126.13800000000002|
|  2|  1|16.322011282850543| -125.9546835180922|
|  3|  1|  16.3627888393447|  -125.771251764291|
|  4|  1|16.403332351824535|-125.58770530205759|
|  5|  1| 16.44364150464794|-125.40404469724042|
+---+---+------------------+-------------------+
only showing top 5 rows



In [11]:
geoHashFunc = udf(lambda x,y: pgh.encode(x,y, precision= 5))
latlon = latlon.withColumn("geoEncode", geoHashFunc(col('Lat'), col('Lon')))

In [12]:
latlon.show(5)

+---+---+------------------+-------------------+---------+
| Nx| Ny|               Lat|                Lon|geoEncode|
+---+---+------------------+-------------------+---------+
|  1|  1|16.280999999999988|-126.13800000000002|    94yk9|
|  2|  1|16.322011282850543| -125.9546835180922|    94ykv|
|  3|  1|  16.3627888393447|  -125.771251764291|    94yt4|
|  4|  1|16.403332351824535|-125.58770530205759|    94ytq|
|  5|  1| 16.44364150464794|-125.40404469724042|    94yvd|
+---+---+------------------+-------------------+---------+
only showing top 5 rows



In [13]:
latlon = latlon.drop("Lat", "Lon")

In [14]:
%%time
centers = spark.read.format('csv')\
        .options(header= 'true', inferSchema= 'true')\
        .load("s3a://partly-cloudy-common-area/proof_of_concept/rapCellNxNy_ARTCC_intersect.csv")

centers.count()

CPU times: user 2.82 ms, sys: 0 ns, total: 2.82 ms
Wall time: 800 ms


25323

In [15]:
centers.groupBy(col("NAME")).agg(count(col("NAME"))).show()

+--------------+-----------+
|          NAME|count(NAME)|
+--------------+-----------+
|  INDIANAPOLIS|        629|
|       ATLANTA|        696|
|         MIAMI|        821|
|       MEMPHIS|        965|
|       OAKLAND|       1203|
|   MINNEAPOLIS|       2734|
|        DENVER|       1806|
|       HOUSTON|       1421|
|  JACKSONVILLE|       1264|
|       BOSTON |       1070|
|   ALBUQUERQUE|       1522|
|    WASHINGTON|        796|
|       CHICAGO|        690|
|   LOS ANGELES|       1157|
|    CLEVELAND |        600|
|       SEATTLE|       1876|
|    FORT WORTH|       1038|
|   KANSAS CITY|       1165|
|      NEW YORK|        932|
|SALT LAKE CITY|       2938|
+--------------+-----------+



In [16]:
centers = centers.drop("IDENT")

In [17]:
%%time
rap = spark.read.format('parquet')\
        .load("s3a://partly-cloudy-rap-parquet/*.parquet")


CPU times: user 1.46 ms, sys: 761 µs, total: 2.22 ms
Wall time: 12.7 s


In [18]:
rap.printSchema()

root
 |-- dateTime: timestamp (nullable = true)
 |-- hPa: integer (nullable = true)
 |-- FLevel: integer (nullable = true)
 |-- Nx: integer (nullable = true)
 |-- Ny: integer (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Lon: double (nullable = true)
 |-- Temperature: double (nullable = true)
 |-- RH_ice: double (nullable = true)
 |-- IsISSR: integer (nullable = true)



In [19]:
%%time
rap_centers = rap.join(centers, ['Nx', 'Ny'], "inner")
rap_centers.count()

CPU times: user 44.3 ms, sys: 29.3 ms, total: 73.6 ms
Wall time: 10min 11s


2864031300

In [20]:
rap_region = rap_centers.filter(col('NAME') == center_name)
rap_region.show()

+---+---+-------------------+---+------+------------------+------------------+-----------+------+------+----------+
| Nx| Ny|           dateTime|hPa|FLevel|               Lat|               Lon|Temperature|RH_ice|IsISSR|      NAME|
+---+---+-------------------+---+------+------------------+------------------+-----------+------+------+----------+
|249| 80|2021-05-21 06:00:00|150|   440| 33.08049989853595|-76.81416143661127|    206.711| 57.43|     0|WASHINGTON|
|247| 81|2021-05-21 06:00:00|150|   440|33.307553271483734|-77.21391541074637|    205.773| 73.12|     0|WASHINGTON|
|248| 81|2021-05-21 06:00:00|150|   440|33.283765319430074|-76.99953666572978|    206.273| 63.44|     0|WASHINGTON|
|249| 81|2021-05-21 06:00:00|150|   440| 33.25969259374498|-76.78524843292448|    206.773|  56.3|     0|WASHINGTON|
|246| 82|2021-05-21 06:00:00|150|   440| 33.51023842897767|-77.40037969646949|    205.336| 82.37|     0|WASHINGTON|
|247| 82|2021-05-21 06:00:00|150|   440| 33.48670942545162| -77.18557750

%%time
rap_flat200430 = rap_region.filter((col("FLevel") >= 200) & (col("FLevel") <= 430))\
                            .groupBy("dateTime", "Nx", "Ny")\
                            .agg(sum("IsISSR").alias("iSSRs200430"),
                                 mean("Temperature").alias("meanTemp200430"),
                                 mean("RH_ice").alias("meanRH_ice200430"))
rap_flat200430.count()

In [21]:
%%time
rap_flat300370 = rap_region.filter((col("FLevel") >= 300) & (col("FLevel") <= 370))\
                            .groupBy("dateTime", "Nx", "Ny")\
                            .agg(sum("IsISSR").alias("iSSRs300370"),
                                 mean("Temperature").alias("meanTemp300370"),
                                 mean("RH_ice").alias("meanRH_ice300370"))
rap_flat300370.count()

CPU times: user 22.3 ms, sys: 6.28 ms, total: 28.6 ms
Wall time: 3min 36s


6925200

In [22]:
centerName = center_name.lower()
path = "s3a://partly-cloudy-common-area/" + centerName + "_flat300370.parquet"
rap_flat300370.write.parquet(path, mode= "overwrite", compression= 'gzip')

%%time
rap_flat_combined = rap_flat200430.join(rap_flat300370, ['dateTime', 'Nx', 'Ny'], "inner")
rap_flat_combined.count()

%%time
rap_flat200430 = rap_flat200430.withColumn("localDateTime", col("dateTime") - expr(time_lag))
rap_flat200430.count()
rap_flat200430 = rap_flat200430.withColumn("dateOnly", to_date(col("localDateTime")))
rap_flat200430 = rap_flat200430.withColumn("hour", hour(col("localDateTime")))
periodFunc = udf(lambda x: x//time_interval * time_interval)
rap_flat200430 = rap_flat200430.withColumn("period", periodFunc(col('hour')))
rap_flat200430.show()

%%time
rap_flat300370 = rap_flat_combined.withColumn("localDateTime", col("dateTime") - expr(time_lag))
rap_flat300370.count()
rap_flat300370 = rap_flat300370.withColumn("dateOnly", to_date(col("localDateTime")))
rap_flat300370 = rap_flat300370.withColumn("hour", hour(col("localDateTime")))
periodFunc = udf(lambda x: x//time_interval * time_interval)
rap_flat300370 = rap_flat300370.withColumn("period", periodFunc(col('hour')))
rap_flat300370.show()

%%time
rap_flat_combined = rap_flat_combined.withColumn("localDateTime", col("dateTime") - expr(time_lag))
rap_flat_combined.count()
rap_flat_combined = rap_flat_combined.withColumn("dateOnly", to_date(col("localDateTime")))
rap_flat_combined = rap_flat_combined.withColumn("hour", hour(col("localDateTime")))
periodFunc = udf(lambda x: x//time_interval * time_interval)
rap_flat_combined = rap_flat_combined.withColumn("period", periodFunc(col('hour')))
rap_flat_combined.show()

centerName = center_name.lower()
path = "s3a://partly-cloudy-ml-inputs-parquet/" + centerName + "_flat200430.parquet"
rap_flat200430.write.parquet(path, mode= "overwrite", compression= 'gzip')

%%time
rap_flat_combined_pd = rap_flat200430.groupBy(col("dateOnly"), col("period"), col("Nx"), col("Ny"))\
                                        .agg(sum("iSSRs200430").alias("pdISSRs200430"),
                                            #sum("iSSRs300370").alias("pdISSRs300370"),
                                            mean("meanTemp200430").alias("pdMeanTemp200430"),
                                            #mean("meanTemp300370").alias("pdMeanTemp300370"),
                                            mean("meanRH_ice200430").alias("pdMeanRH_ice200430")
                                            #mean("meanRH_ice300370").alias("pdMeanRH_ice300370")
                                            )

rap_flat_combined_pd.count()

%%time
rap_flat_combined_pd = rap_flat300370.groupBy(col("dateOnly"), col("period"), col("Nx"), col("Ny"))\
                                        .agg(#sum("iSSRs200430").alias("pdISSRs200430"),
                                            sum("iSSRs300370").alias("pdISSRs300370"),
                                            #mean("meanTemp200430").alias("pdMeanTemp200430"),
                                            mean("meanTemp300370").alias("pdMeanTemp300370"),
                                            #mean("meanRH_ice200430").alias("pdMeanRH_ice200430"),
                                            mean("meanRH_ice300370").alias("pdMeanRH_ice300370")
                                            )

rap_flat_combined_pd.count()

centerName = center_name.lower()
path = "s3a://partly-cloudy-common-area/" + centerName + "_6_hour_flat300370.parquet"
rap_flat_combined_pd.write.parquet(path, mode= "overwrite", compression= 'gzip')

%%time
centerName = center_name.lower()
rap_flat_combined_pd = spark.read.format('parquet')\
                            .load("s3a://partly-cloudy-common-area/" + centerName + "_6_hour_flat.parquet")


%%time
rap_flat_combined_pdPandas = rap_flat_combined_pd.toPandas()
rap_flat_combined_pdPandas

%%time
rap_flat_combined_pdPandas['LocalDateTimePeriod'] = [pd.Timestamp(rap_flat_combined_pdPandas['dateOnly'][i]) + timedelta(hours= int(rap_flat_combined_pdPandas['period'][i])) for i in range(len(rap_flat_combined_pdPandas))]
rap_flat_combined_pdPandas

%%time
rap_flat_combined_pdPandas.drop(columns= ['dateOnly', 'period'], inplace= True)

%%time
centers_df = wr.s3.read_csv('s3://partly-cloudy-common-area/proof_of_concept/rapCellNxNy_ARTCC_intersect.csv')
centers_df

%%time
latlon_df = wr.s3.read_csv('s3://partly-cloudy-common-area/proof_of_concept/latlonRAP.csv')
latlon_df

%%time
center_flat_pdPandas_join = pd.merge(rap_flat_combined_pdPandas, centers_df, on= ['Nx', 'Ny'])
center_flat_pdPandas_join = pd.merge(center_flat_pdPandas_join, latlon_df, on= ['Nx', 'Ny'])
center_flat_pdPandas_join

%%time
center_flat_pdPandas_join['geoEncode'] = [pgh.encode(center_flat_pdPandas_join['Lat'][i], center_flat_pdPandas_join['Lon'][i], precision= 5) for i in range(len(center_flat_pdPandas_join))]
center_flat_pdPandas_join

%%time
center_flat_pd_final = center_flat_pdPandas_join[['LocalDateTimePeriod', 
                                                  'Nx', 'Ny', 
                                                  'Lat', 'Lon', 
                                                  'geoEncode', 
                                                  'NAME', 
                                                  'pdISSRs200430', 
                                                  'pdISSRs300370',
                                                  'pdMeanTemp200430',
                                                  'pdMeanTemp300370',
                                                  'pdMeanRH_ice200430',
                                                  'pdMeanRH_ice300370']].copy()
center_flat_pd_final

output_bucket = 'partly-cloudy-ml-inputs'
outputFileName = center_name + '_6_hour_flat.csv' 

wr.s3.to_csv(center_flat_pd_final, f"s3://{output_bucket}/{outputFileName}", index=False)